## Importação de bibliotecas

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


import time
import pandas as pd
import random
from datetime import datetime


## Importação e tratamento da base

In [20]:
base = pd.read_excel('base.xlsx')
base.columns = base.iloc[2]
base = base[3:]
base = base.reset_index(drop=True)

# display(base)
contatados = pd.DataFrame(columns = ['NOME NO CELULAR', 'TELEFONE', 'STATUS WHATSAPP'])
status = base.loc[0,'STATUS WHATSAPP']
display(base)

2,CPF / CNPJ,TELEFONES,EMAIL,NOME NO CELULAR,NOME,STATUS WHATSAPP,STATUS EMAIL
0,94748314834,"(11) 993421808,(11) 930890404,",0,94748314834ANTONIO VICENTE DE MELO,ANTONIO,NaN,NaN
1,94695270897,"(11) 972571507,(11) 42442005,(11) 934332845",ANA.GOAES@GLOBO.COM,94695270897HELIODORIO DE SOUZA GOES,HELIODORIO,NaN,NaN
2,89103130800,"(11) 967778397,(11) 58248963,",0,89103130800ADEACIR MATIAS RODRIGUES,ADEACIR,NaN,NaN
3,86910639472,"(34) 992695531,(11) 979936226,(87) 37731515",MARIAJOSE.FRANCA.963@GMAIL.COM,86910639472MARIA JOSE FRANCA DE MELO,MARIA,NaN,NaN
4,86627180430,"(11) 987921922,,",0,86627180430SEVERINA SILVA CAVALCANTE DOS SANTOS,SEVERINA,NaN,NaN
...,...,...,...,...,...,...,...
146,407369899,"(11) 987823137,(11) 982623860,",0,407369899EVANDO HERCOLANO DE OLIVEIRA,EVANDO,NaN,NaN
147,334096847,"(11) 979630885,,",ADEMAR_RODRIGUES58@OUTLOOK.COM,334096847ADEMAR RODRIGUES BITENCOURT,ADEMAR,NaN,NaN
148,307359808,"(11) 42032995,(11) 964010061,",MARLY.FELIPE@TERRA.COM.BR,307359808MARLY FELIPE DA SILVA BATISTA,MARLY,NaN,NaN
149,107178893,"(11) 939456239,(11) 42032727,(11) 58912276",CARDOZOV57@GMAIL.COM,107178893VERA LUCIA CARDOZO,VERA,NaN,NaN


## Abertura de navegador

In [12]:
servico = Service(version= '122')
caminho_perfil = r'C:\Users\serci\AppData\Local\Google\Chrome\User Data\Profile Selenium'
options = webdriver.ChromeOptions()
options.add_argument(f'user-data-dir={caminho_perfil}')

navegador = webdriver.Chrome(service=servico, options = options)
navegador.get('https://web.whatsapp.com/')
while len(navegador.find_elements(By.CLASS_NAME, 'selectable-text')) == 0:
    time.sleep(1)

## Definição de funções

In [13]:



def procura_contato(nome, telefone):
    campo_de_busca = navegador.find_element(By.CLASS_NAME, 'selectable-text')
    campo_de_busca.send_keys(Keys.CONTROL, 'a')
    campo_de_busca.send_keys(telefone)
    time.sleep(1)
    contato = navegador.find_elements(By.CLASS_NAME, '_21S-L')
    
    mensagem = f'''
Cliente {nome}, somos da CAIXA - Agência Santo Eduardo do Embu, tudo bem?
#quebra#
Você que é beneficiário do BPC pelo INSS, faça o desbloqueio do seu benefício no INSS e aproveite para fazer seu crédito consignado CAIXA com as menores taxas do  mercado.     
#quebra#                                    
Além disso, você pode ter um cartão de crédito CAIXA sem anuidade e com juros reduzidos.
#quebra#
Caso tenha interesse, responda esta mensagem ou compareça diretamente na agência sem pegar fila procurando por nossos gerentes Níger ou Aldecir.
#quebra#
Atenciosamente CAIXA - Agência Santo Eduardo do Embu - Av. João Paulo II, 1344 - Jardim Santa Tereza - Embu das Artes - SP 
#quebra#
Lembrando que em breve estaremos novamente no bairro do Santo Eduardo, próximo à Drogaria São Paulo.
'''
    if len(contato)> 0:
        contato[0].click()
        envia_mensagem(mensagem)
        retorna_status('enviado, ')
        return
    else:
        retorna_status('tel sem whats,')

def horario():
    # Obtenha o horário atual
    horario_atual = datetime.now()
    # Imprima o horário atual
    return horario_atual.strftime('%H:%M')

def envia_mensagem(texto):
    secciona_mensagem = texto.split('#quebra#')
    for pedaco in secciona_mensagem:
        time.sleep(0.7)
        navegador.find_element(By.CLASS_NAME, '_3Uu1_').send_keys(pedaco, Keys.ENTER)
        time.sleep(1.5)   
        time.sleep(random.randrange(1,6,1))
def retorna_status(texto):
    status = base.loc[cliente, 'STATUS WHATSAPP']
    if pd.isna(status):
        status = texto
    else:
        status = status + texto
    base.loc[cliente, 'STATUS WHATSAPP'] = status
    

## Varrer a base enviando mensagens

In [14]:
contador = 0

print(f'inicio em {horario()}')
for cliente in base.index:
    contador += 1
    nome = base.loc[cliente, 'NOME']
    lista_telefones = base.loc[cliente,'TELEFONES'].split(',')
    for telefone in lista_telefones:
        if telefone != '':
            procura_contato(nome, telefone)
    if contador % 5 == 0:
        contador = 0
        base.to_excel('Retorno do envio.xlsx', index = False)
print(f'concluido as {horario()}')


inicio em 00:15
concluido as 00:17


## Salva retorno da base

In [19]:
base.to_excel('Retorno do envio.xlsx', index=False)

2,CPF / CNPJ,TELEFONES,EMAIL,NOME NO CELULAR,NOME,STATUS WHATSAPP,STATUS EMAIL
0,NaN,"55415865, 4254854445, 11982662870",NaN,NaN,sergio pj,"enviado, tel sem whats,enviado,",NaN
1,NaN,"43215432, 11951496374",NaN,NaN,Yasminzinha Mini,"tel sem whats,tel sem whats",NaN
2,NaN,"23456789, 11981317092",NaN,NaN,Rog gério,"tel sem whats,enviado,",NaN
